#### Imports

In [14]:
# Manipulação de dados
import pandas as pd
pd.set_option('display.max_columns', None)

# Estética do Código
import os

# Trabalhar com tempo no código
from time import sleep

# Display
import ipywidgets as widgets
from IPython.display import display, clear_output

### Análises

#### Análise 1: Qual candidato foi mais votado no município X?

In [19]:
# Dados
municipios = pd.read_excel('perfil_municipio.xlsx')
candidatos = pd.read_excel('SPturno1_candidatos.xlsx')
cargos = pd.read_excel('SPturno1_cargos.xlsx')
resultado = pd.read_csv('SPturno1_clean.csv', usecols=['CD_MUNICIPIO', 'CD_CARGO_PERGUNTA', 'ID_CANDIDATO', 'QT_COMPARECIMENTO'])

# Excluir linhas onde ID_CANDIDATO é igual a 0 ou 7
resultado = resultado[~resultado['ID_CANDIDATO'].isin([0, 7])]

# JOIN com 'municipios' para obter informações sobre os municípios
merge_municipios = resultado.merge(municipios, left_on='CD_MUNICIPIO', right_on='CD_MUNICIPIO')

# JOIN com 'candidatos' para obter informações sobre os candidatos
merge_completo = merge_municipios.merge(candidatos, left_on='ID_CANDIDATO', right_on='ID_CANDIDATO')

# Agrupar por nome de município e cargo, e calcular a soma dos votos por código de candidato
grouped = merge_completo.groupby(['NM_MUNICIPIO', 'DS_CARGO_PERGUNTA', 'NM_CANDIDATO'])['QT_COMPARECIMENTO'].sum().reset_index()

# Encontrar o candidato mais votado para cada grupo
idx_max_votos = grouped.groupby(['NM_MUNICIPIO', 'DS_CARGO_PERGUNTA'])['QT_COMPARECIMENTO'].idxmax()
candidatos_mais_votados = grouped.loc[idx_max_votos]

In [18]:
# Função para atualizar a lista de municípios com base no texto digitado
def update_municipios(change):
    selected_text = change.new.lower()
    if selected_text:
        filtered_municipios = candidatos_mais_votados[candidatos_mais_votados['NM_MUNICIPIO'].str.lower().str.startswith(selected_text)]
        municipios_dropdown.options = filtered_municipios['NM_MUNICIPIO'].unique().tolist()
    else:
        municipios_dropdown.options = candidatos_mais_votados['NM_MUNICIPIO'].unique().tolist()

# Função para exibir os resultados filtrados
def display_filtered_results(selected_municipio):
    clear_output(wait=True)  # Limpar saída anterior
    filtered_results = candidatos_mais_votados[candidatos_mais_votados['NM_MUNICIPIO'] == selected_municipio]
    display(filtered_results)

# Criar caixa de texto interativa
search_box = widgets.Text(
    value='',
    placeholder='Digite o nome do município...',
    description='Pesquisar:',
    disabled=False
)

# Atualizar a lista de municípios quando o texto for digitado
search_box.observe(update_municipios, names='value')

# Criar dropdown para exibir os municípios filtrados
municipios_dropdown = widgets.Dropdown(
    options=candidatos_mais_votados['NM_MUNICIPIO'].unique().tolist(),
    description='Município:'
)

# Exibir caixa de texto e dropdown
display(search_box)

# Atualizar os resultados filtrados quando o usuário selecionar um município no dropdown
widgets.interact(display_filtered_results, selected_municipio=municipios_dropdown)


Text(value='', description='Pesquisar:', placeholder='Digite o nome do município...')

interactive(children=(Dropdown(description='Município:', options=('ADAMANTINA', 'ADOLFO', 'AGUAÍ', 'AGUDOS', '…

<function __main__.display_filtered_results(selected_municipio)>

#### Análise 2: Qual município o candidato X foi mais votado?

In [21]:
# Dados
municipios = pd.read_excel('perfil_municipio.xlsx')
candidatos = pd.read_excel('SPturno1_candidatos.xlsx')
cargos = pd.read_excel('SPturno1_cargos.xlsx')
resultado = pd.read_csv('SPturno1_clean.csv', usecols=['CD_MUNICIPIO', 'CD_CARGO_PERGUNTA', 'ID_CANDIDATO', 'QT_COMPARECIMENTO'])

# Realizar o groupby e a agregação
grouped = merge_completo.groupby(['NM_CANDIDATO', 'NM_MUNICIPIO'])['QT_COMPARECIMENTO'].sum().reset_index()

# Encontrar o candidato mais votado para cada candidato
idx_max_votos = grouped.groupby(['NM_CANDIDATO'])['QT_COMPARECIMENTO'].idxmax()
candidatos_mais_votados = grouped.loc[idx_max_votos]


                     NM_CANDIDATO       NM_MUNICIPIO  QT_COMPARECIMENTO
0                  A C CALCULISTA            TAUBATÉ              70140
1                     A CASSIMIRO        SERTÃOZINHO              16248
2      A DONA ONÇA DOS BRINQUEDOS       PITANGUEIRAS               7068
3                      A MARICATO       ITAPETININGA               2425
4                    AARON TRESNY           VALINHOS               7638
...                           ...                ...                ...
90928                 ÍTALA LAGES               IPUÃ               4764
90929             ÍTALO CAVENAGHI           BARRETOS              12841
90930           ÍTALO DA MARABRAZ            CAJAMAR              28746
90931             ÍTALO GONÇALVES  BRAGANÇA PAULISTA              19098
90932               ÍTALO MOREIRA           SOROCABA             272992

[73824 rows x 3 columns]


In [22]:
# Função para atualizar a lista de candidatos com base no texto digitado
def update_candidatos(change):
    selected_text = change.new.lower()
    if selected_text:
        filtered_candidatos = candidatos_mais_votados[candidatos_mais_votados['NM_CANDIDATO'].str.lower().str.startswith(selected_text)]
        candidatos_dropdown.options = filtered_candidatos['NM_CANDIDATO'].unique().tolist()
    else:
        candidatos_dropdown.options = candidatos_mais_votados['NM_CANDIDATO'].unique().tolist()

# Função para exibir os resultados filtrados
def display_filtered_results(selected_candidato):
    clear_output(wait=True)  # Limpar saída anterior
    filtered_results = candidatos_mais_votados[candidatos_mais_votados['NM_CANDIDATO'] == selected_candidato]
    display(filtered_results)

# Criar caixa de texto interativa
search_box = widgets.Text(
    value='',
    placeholder='Digite o nome do candidato...',
    description='Pesquisar:',
    disabled=False
)

# Atualizar a lista de candidatos quando o texto for digitado
search_box.observe(update_candidatos, names='value')

# Criar dropdown para exibir os candidatos filtrados
candidatos_dropdown = widgets.Dropdown(
    options=candidatos_mais_votados['NM_CANDIDATO'].unique().tolist(),
    description='Candidato:'
)

# Exibir caixa de texto e dropdown
display(search_box)

# Atualizar os resultados filtrados quando o usuário selecionar um candidato no dropdown
widgets.interact(display_filtered_results, selected_candidato=candidatos_dropdown)


Text(value='', description='Pesquisar:', placeholder='Digite o nome do candidato...')

interactive(children=(Dropdown(description='Candidato:', options=('A C CALCULISTA', 'A CASSIMIRO', 'A DONA ONÇ…

<function __main__.display_filtered_results(selected_candidato)>